# MotionCommander Basics

This tutorial notebook introduces the basics of Bitcraze's MotionCommander library. MotionCommander is an abstraction library that makes writing Crazyflie movement scripts easy. 

MotionCommander requires some sort of positioning hardware support such as the Flow deck. The library does not have a notion of absolute position (such as GPS) and instead uses velocity setpoints. Due to this, the error in position will accumulate over time.

The API contains a set of primitives that are easy to understand and use, such as “go forward” or “turn around”.

There are two flavors of primitives:
1. Actions that are blocking and return when a motion is completed
2. Actions that starts a motion and return immediately

In the second variation the user has to stop or change the motion when appropriate by issuing new commands.

The MotionCommander can be used as context manager using the with keyword. In this mode of operation takeoff and landing is executed when the context is created/closed.

Reference Documentation: 
* [Step-by-Step: Motion Commander](https://www.bitcraze.io/documentation/repository/crazyflie-lib-python/master/user-guides/sbs_motion_commander/)
* [MotionCommander GitHub](https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/step-by-step/sbs_motion_commander.py)


## Imports

In [ ]:
# Standard imports
import logging
import sys
import time
from threading import Event
from dotenv import load_dotenv
import os


# Crazyflie imports
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper

### Global Variables & Configs

In [39]:
load_dotenv()  # take environment variables from .env

# Your radio frequency. This can be found in the Crazyflie client by running "cfclient" in your terminal.
URI = uri_helper.uri_from_env(default=os.getenv("RADIO_URI"))

# --- Global variables
DEFAULT_HEIGHT = 1.0
BOX_LIMIT = 0.5

# Event to signal that the flow deck is attached
deck_attached_event = Event()

# Variable to store the (x, y) position estimates used by the logger
position_estimate = [0, 0]

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)

In [40]:
def param_deck_flow(_, value_str):
    """
    Function to check if the flow deck is attached to the drone. 
    Note: that the value type that the param_deck_flow() is a string type, so we convert it to an int before operations.
    All deck values are here: https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/api/params/#deck
    """
    value = int(value_str)
    print(value)
    if value:
        deck_attached_event.set()
        print('Deck is attached!')
    else:
        print('Deck is NOT attached!')


def take_off_simple(scf):
    """
    Simple take off and land function using the MotionCommander class.
    1. Take off to a default height (defined in the global variable DEFAULT_HEIGHT)
    2. Move up 0.3m
    3. Hover for 3 seconds
    4. Land
    """
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        # Take off to the default height and then go up 0.3m
        mc.up(0.3)
        time.sleep(3)
        mc.stop()
        

def move_linear_simple(scf):
    """
    Simple linear movement function using the MotionCommander class.
    1. Take off to a default height (defined in the global variable DEFAULT_HEIGHT)
    2. Move forward 0.5m
    3. Move backward 0.5m
    4. Land
    """
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        time.sleep(1)
        mc.forward(0.5)
        time.sleep(1)
        mc.back(0.5)
        time.sleep(1)


def log_pos_callback(timestamp, data, logconf):
    # Prints logged X, Y position estimates
    print(data)



In [41]:
if __name__ == '__main__':
    cflib.crtp.init_drivers()
    
        
    with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Callback to check if the flow deck is attached
        scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)

        # Log event actions
        logconf = LogConfig(name='Position', period_in_ms=10)
        logconf.add_variable('stateEstimate.x', 'float')
        logconf.add_variable('stateEstimate.y', 'float')
        scf.cf.log.add_config(logconf)
        logconf.data_received_cb.add_callback(log_pos_callback)
        
        
        # Wait until the flow deck is attached (max wait time is 5 seconds)
        if not deck_attached_event.wait(timeout=5):
            print('No flow deck detected!')
            sys.exit(1)
            
        logconf.start()
            
        # Arm the Crazyflie
        scf.cf.platform.send_arming_request(True)
        time.sleep(1.0)
        move_linear_simple(scf)
        
        logconf.stop()


1
Deck is attached!
{'stateEstimate.x': -72.5745620727539, 'stateEstimate.y': -48.112850189208984}
{'stateEstimate.x': -72.60079193115234, 'stateEstimate.y': -48.149375915527344}
{'stateEstimate.x': -72.62702941894531, 'stateEstimate.y': -48.1859130859375}
{'stateEstimate.x': -72.64923858642578, 'stateEstimate.y': -48.21603012084961}
{'stateEstimate.x': -72.67547607421875, 'stateEstimate.y': -48.252586364746094}
{'stateEstimate.x': -72.70172119140625, 'stateEstimate.y': -48.289154052734375}
{'stateEstimate.x': -72.72796630859375, 'stateEstimate.y': -48.32572937011719}
{'stateEstimate.x': -72.75618743896484, 'stateEstimate.y': -48.36549377441406}
{'stateEstimate.x': -72.78244018554688, 'stateEstimate.y': -48.40209197998047}
{'stateEstimate.x': -72.80870056152344, 'stateEstimate.y': -48.43870162963867}
{'stateEstimate.x': -72.83496856689453, 'stateEstimate.y': -48.475318908691406}
{'stateEstimate.x': -72.86495971679688, 'stateEstimate.y': -48.5179443359375}
{'stateEstimate.x': -72.891235